# Poltician's education pathways

Only current-ish data https://www.smh.com.au/interactive/2021/careers-before-politics/

TODO:
- What is a senior MP ?
- Degree types, undergrad,

In [1]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd

endpoint_url = "https://query.wikidata.org/sparql"

query_reps = """SELECT ?item ?itemLabel ?group ?groupLabel ?district ?districtLabel ?term ?termLabel ?edu ?eduLabel ?start ?end
WHERE
{
  ?item p:P39 ?statement .
  ?statement ps:P39/wdt:P279* wd:Q18912794 ; pq:P580 ?start .
  OPTIONAL { ?statement pq:P2937 ?term }
  OPTIONAL { ?statement pq:P582  ?end }
  OPTIONAL { ?statement pq:P768  ?district }
  OPTIONAL { ?statement pq:P4100 ?group }
  OPTIONAL { ?item wdt:P69 ?edu .
             ?edu rdfs:label ?eduLabel FILTER (lang(?eduLabel) = "en")}
  FILTER(!BOUND(?end) || ?end > NOW())
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
ORDER BY ?start ?end"""

query_senate = """
SELECT ?item ?itemLabel ?group ?groupLabel ?district ?districtLabel ?term ?termLabel ?edu ?eduLabel ?start ?end
WHERE
{
  ?item p:P39 ?statement .
  ?statement ps:P39/wdt:P279* wd:Q6814428 ; pq:P580 ?start .
  OPTIONAL { ?statement pq:P2937 ?term }
  OPTIONAL { ?statement pq:P582  ?end }
  OPTIONAL { ?statement pq:P768  ?district }
  OPTIONAL { ?statement pq:P4100 ?group }
  OPTIONAL { ?item wdt:P69 ?edu .
             ?edu rdfs:label ?eduLabel FILTER (lang(?eduLabel) = "en")}
  FILTER(!BOUND(?end) || ?end > NOW())
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
ORDER BY ?start ?end
"""


def get_results(endpoint_url, query):
    user_agent = "WDQS Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


In [2]:
def clean_results(results:dict) -> pd.DataFrame:
    output = pd.json_normalize(results)
    col_vals = [ c for c in output.columns if c.endswith(".value")]
    output_cleaned = output[col_vals]

    output_cleaned.columns = [ c.replace(".value", "") for c in col_vals ]
    return output_cleaned

In [3]:
senate_results = get_results(endpoint_url, query_senate)
senate_results

{'head': {'vars': ['item',
   'itemLabel',
   'group',
   'groupLabel',
   'district',
   'districtLabel',
   'term',
   'termLabel',
   'edu',
   'eduLabel',
   'start',
   'end']},
 'results': {'bindings': [{'item': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q16191011'},
    'edu': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q7865388'},
    'eduLabel': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'UNSW Faculty of Law'},
    'start': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime',
     'type': 'literal',
     'value': '1997-04-09T00:00:00Z'},
    'district': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q56649105'},
    'group': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q1065320'},
    'itemLabel': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'Marise Payne'},
    'groupLabel': {'xml:lang': 'en', 'type': 'literal', 'value': 'Coalition'},
    'districtLabel': {'xml:lang': 'en',
     'ty

In [4]:
senate_df = clean_results(senate_results['results']['bindings'])
senate_df

,item,edu,eduLabel,start,district,group,itemLabel,groupLabel,districtLabel
0,http://www.wikidata.org/entity/Q16191011,http://www.wikidata.org/entity/Q7865388,UNSW Faculty of Law,1997-04-09T00:00:00Z,http://www.wikidata.org/entity/Q56649105,http://www.wikidata.org/entity/Q1065320,Marise Payne,Coalition,New South Wales
1,http://www.wikidata.org/entity/Q16191011,http://www.wikidata.org/entity/Q18545951,University of New South Wales Law School,1997-04-09T00:00:00Z,http://www.wikidata.org/entity/Q56649105,http://www.wikidata.org/entity/Q1065320,Marise Payne,Coalition,New South Wales
2,http://www.wikidata.org/entity/Q456759,http://www.wikidata.org/entity/Q15574,University of Adelaide,2002-07-01T00:00:00Z,http://www.wikidata.org/entity/Q56649110,http://www.wikidata.org/entity/Q216082,Penny Wong,Australian Labor Party,South Australia
3,http://www.wikidata.org/entity/Q456759,http://www.wikidata.org/entity/Q15576,University of South Australia,2002-07-01T00:00:00Z,http://www.wikidata.org/entity/Q56649110,http://www.wikidata.org/entity/Q216082,Penny Wong,Australian Labor Party,South Australia
4,http://www.wikidata.org/entity/Q16189453,http://www.wikidata.org/entity/Q962011,University of Tasmania,2005-07-01T00:00:00Z,http://www.wikidata.org/entity/Q56649108,http://www.wikidata.org/entity/Q216082,Helen Polley,Australian Labor Party,Tasmania
...,...,...,...,...,...,...,...,...,...
114,http://www.wikidata.org/entity/Q112152550,NaN,NaN,2022-07-01T00:00:00Z,http://www.wikidata.org/entity/Q56649104,http://www.wikidata.org/entity/Q216082,Fatima Payman,Australian Labor Party,Western Australia
115,http://www.wikidata.org/entity/Q112581963,NaN,NaN,2022-07-01T00:00:00Z,http://www.wikidata.org/entity/Q56649110,http://www.wikidata.org/entity/Q1065320,Kerrynne Liddle,Coalition,South Australia
116,http://www.wikidata.org/entity/Q112642694,NaN,NaN,2022-07-01T00:00:00Z,http://www.wikidata.org/entity/Q56649112,http://www.wikidata.org/entity/Q15130081,Ralph Babet,United Australia Party,Victoria
117,http://www.wikidata.org/entity/Q16185446,http://www.wikidata.org/entity/Q1375146,Murdoch University,2022-07-26T00:00:00Z,NaN,NaN,Sue Lines,NaN,NaN


In [5]:
reps_results  = get_results(endpoint_url, query_reps)
reps_df = clean_results(reps_results['results']['bindings'])
reps_df

,item,edu,eduLabel,start,district,group,itemLabel,groupLabel,districtLabel
0,http://www.wikidata.org/entity/Q4932983,http://www.wikidata.org/entity/Q866012,University of Queensland,1993-03-13T00:00:00Z,http://www.wikidata.org/entity/Q2973687,http://www.wikidata.org/entity/Q6378340,Bob Katter,Katter's Australian Party,Kennedy
1,http://www.wikidata.org/entity/Q335697,http://www.wikidata.org/entity/Q487556,University of Sydney,1996-03-02T00:00:00Z,http://www.wikidata.org/entity/Q1079489,http://www.wikidata.org/entity/Q216082,Anthony Albanese,Australian Labor Party,Grayndler
2,http://www.wikidata.org/entity/Q335697,http://www.wikidata.org/entity/Q7896374,University of Sydney Business School,1996-03-02T00:00:00Z,http://www.wikidata.org/entity/Q1079489,http://www.wikidata.org/entity/Q216082,Anthony Albanese,Australian Labor Party,Grayndler
3,http://www.wikidata.org/entity/Q335697,http://www.wikidata.org/entity/Q7594250,St Mary's Cathedral College,1996-03-02T00:00:00Z,http://www.wikidata.org/entity/Q1079489,http://www.wikidata.org/entity/Q216082,Anthony Albanese,Australian Labor Party,Grayndler
4,http://www.wikidata.org/entity/Q7684036,http://www.wikidata.org/entity/Q741082,Macquarie University,1998-10-03T00:00:00Z,http://www.wikidata.org/entity/Q1075639,http://www.wikidata.org/entity/Q216082,Tanya Plibersek,Australian Labor Party,Sydney
...,...,...,...,...,...,...,...,...,...
256,http://www.wikidata.org/entity/Q112127622,NaN,NaN,2022-05-21T00:00:00Z,http://www.wikidata.org/entity/Q2973656,http://www.wikidata.org/entity/Q1065320,Jenny Ware,Coalition,Hughes
257,http://www.wikidata.org/entity/Q112131017,NaN,NaN,2022-05-21T00:00:00Z,http://www.wikidata.org/entity/Q107393285,http://www.wikidata.org/entity/Q216082,Sam Rae,Australian Labor Party,Hawke
258,http://www.wikidata.org/entity/Q112134180,NaN,NaN,2022-05-21T00:00:00Z,http://www.wikidata.org/entity/Q2973472,http://www.wikidata.org/entity/Q1065320,Henry Pike,Coalition,Bowman
259,http://www.wikidata.org/entity/Q112567447,NaN,NaN,2022-05-21T00:00:00Z,http://www.wikidata.org/entity/Q2973800,http://www.wikidata.org/entity/Q216082,Andrew Charlton,Australian Labor Party,Parramatta


In [6]:
edu_cols = [["edu" "eduLabel"]]
education_df : pd.DataFrame = pd.concat([reps_df[["edu","eduLabel"]], senate_df[["edu","eduLabel"]]])
education_df.dropna(inplace=True)
education_df.drop_duplicates(["edu"], inplace=True)
education_df.sort_values(["eduLabel"], inplace=True)

In [13]:
education_df["is_university"] = education_df["eduLabel"].str.contains("University")
high_school_college =pattern = r"(High School|College|Grammar School)"
education_df["is_high_school"] = education_df["eduLabel"].str.contains(high_school_college)
education_df["is_alt_edu"] = (~education_df["is_university"])& (~education_df["is_high_school"])

# todo: high schools non-government, public, both, other

/var/folders/nd/k10412g92s1bk7rll01d59y40000gn/T/ipykernel_67125/4238359028.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  education_df["is_high_school"] = education_df["eduLabel"].str.contains(high_school_college)


In [8]:
education_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 139 entries, 104 to 153
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   edu            139 non-null    object
 1   eduLabel       139 non-null    object
 2   is_university  139 non-null    bool  
dtypes: bool(1), object(2)
memory usage: 3.4+ KB


In [25]:
schools_sparql_entities = education_df[education_df["is_high_school"]].edu.str.extract(r"(Q\d+)")
schools_sparql_entities.columns = ["entity"]
cleaned_ents = ' '.join(['wd:'+ e for e in schools_sparql_entities["entity"].values])
cleaned_ents

'wd:Q4763439 wd:Q4782971 wd:Q4792525 wd:Q4968616 wd:Q4997498 wd:Q17514127 wd:Q5031258 wd:Q5032517 wd:Q5059079 wd:Q5060585 wd:Q6765675 wd:Q5130089 wd:Q5193900 wd:Q5244511 wd:Q5244515 wd:Q5267826 wd:Q6874499 wd:Q5273786 wd:Q5303222 wd:Q5373399 wd:Q5436281 wd:Q5528459 wd:Q5587183 wd:Q5874683 wd:Q6065569 wd:Q6265061 wd:Q6413539 wd:Q6481599 wd:Q6516506 wd:Q6721977 wd:Q6765669 wd:Q6772346 wd:Q1914698 wd:Q6801996 wd:Q6811796 wd:Q6813902 wd:Q6900526 wd:Q6911446 wd:Q14935237 wd:Q6961407 wd:Q1376987 wd:Q62516299 wd:Q7066890 wd:Q7110892 wd:Q7164606 wd:Q7244458 wd:Q7335025 wd:Q7355277 wd:Q2171074 wd:Q7435507 wd:Q14934996 wd:Q7569827 wd:Q7593812 wd:Q7594250 wd:Q7595311 wd:Q7659840 wd:Q7659951 wd:Q48769587 wd:Q7739701 wd:Q7762626 wd:Q7796467 wd:Q7830279 wd:Q7894929 wd:Q7897033 wd:Q7930877 wd:Q7975377 wd:Q8043217 wd:Q8060904'

In [27]:

school_location_query = f"""SELECT ?school ?schoolLabel ?location ?locationLabel
WHERE
{{
  VALUES ?school {{ {cleaned_ents} }}
  ?school wdt:P625 ?location.
  SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
}}"""

school_locs = get_results(endpoint_url, school_location_query)


In [28]:
school_locs

{'head': {'vars': ['school', 'schoolLabel', 'location', 'locationLabel']},
 'results': {'bindings': [{'school': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q1376987'},
    'location': {'datatype': 'http://www.opengis.net/ont/geosparql#wktLiteral',
     'type': 'literal',
     'value': 'Point(-1.25166667 51.75416667)'},
    'schoolLabel': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'New College'},
    'locationLabel': {'type': 'literal',
     'value': 'Point(-1.25166667 51.75416667)'}},
   {'school': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q1914698'},
    'location': {'datatype': 'http://www.opengis.net/ont/geosparql#wktLiteral',
     'type': 'literal',
     'value': 'Point(116.039 -32.0067)'},
    'schoolLabel': {'xml:lang': 'en',
     'type': 'literal',
     'value': 'Mazenod College, Perth'},
    'locationLabel': {'type': 'literal', 'value': 'Point(116.039 -32.0067)'}},
   {'school': {'type': 'uri',
     'value': 'http://www.wikida

In [31]:
careers_csv = pd.read_csv("https://www.smh.com.au/interactive/2021/careers-before-politics/careers.csv")
inc_mat_csv = pd.read_csv("https://www.smh.com.au/interactive/2021/careers-before-politics/incMat.csv")
ministry_csv = pd.read_csv("https://www.smh.com.au/interactive/2021/careers-before-politics/Ministry.csv")

In [32]:
inc_mat_csv

,Member,Party,Sex,Start,Public,Home-schooled,Did not graduate,Non-government,International,Undergraduate,...,Lobbying or Activism,Farming,Academia,Judicial,Nonprofits,Arts,Religion,Local government,State or Territory government,Parliament
0,Anthony Albanese,Labor,M,True,False,False,False,True,False,True,...,False,False,False,False,False,False,False,False,False,True
1,John Alexander,Liberal,M,True,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,True
2,Katie Allen,Liberal,F,True,False,False,False,True,False,True,...,False,False,True,False,True,False,False,False,False,True
3,Anne Aly,Labor,F,True,False,False,False,True,False,True,...,False,False,True,False,False,False,False,False,False,True
4,Karen Andrews,Liberal,F,True,True,False,False,True,False,True,...,True,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,Jess Walsh,Labor,F,True,False,False,False,True,False,True,...,False,False,True,False,False,False,False,False,False,True
223,Larissa Waters,Greens,F,True,True,False,False,False,False,True,...,False,False,False,False,True,False,False,False,False,True
224,Murray Watt,Labor,M,True,True,False,False,False,False,True,...,False,False,False,True,False,False,False,False,True,True
225,Peter Whish-Wilson,Greens,M,True,True,False,False,True,False,True,...,False,False,True,False,False,False,False,False,False,True
